\section{\centering Introdução}

O Spotify é um serviço digital que dá acesso instantâneo a milhões de músicas, podcasts, vídeos e outros conteúdos de criadores no mundo todo. Pode ser acessado pelo *browser*, sendo possível também baixar o aplicativo, estando disponível para diversas plataformas digitais. 
Nesse ensaio, vamos analisar os *reviews* feitos na Play Store sobre a versão para *Android*, dessa maneira vamos examinar o comportamento dos reviews (estrelas), junto com o texto deixado pelo autor do mesmo.


\subsection{Uma visão Geral das Avaliações}

Como é de conhecimento geral, erros geram muito mais comentários que acertos^[Se você não dispõe de tal conhecimento, essa análise é a evidência pura e concentra disso.], dessa maneira, reclamações de problemas tendem a criar mais agitações que congratulações de acertos, nesse sentido, espera-se uma quantidade considerável de avaliações nota 1 do aplicativo. Pois os usuários de aplicativos são propensos a ter expectativas de sua utilização, entretanto, expectativas não satisfeitas, geram frustrações, e uma maneira de destilar tamanha frustração é lançar toda a fúria no *review* na Play Store. Começamos, portanto, avaliando pela @fig-prop, a proporção das avaliações.


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import RocCurveDisplay
from sklearn.linear_model import LogisticRegression
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from utils.utils import time_plot, total_variable, my_wc, Display
prm={"figure.figsize":(9, 3),
'figure.dpi':150}
plt.rcParams.update(prm)
sns.set_palette("viridis")
sns.set_style("whitegrid")

In [ ]:
df = pd.read_csv("data/reviews.csv")

In [ ]:
#| label: fig-prop
#| fig-cap: Proporção das Avaliações por Estrelas
grouped_data = df.groupby("Rating", as_index=False).size()
grouped_data = grouped_data.assign(Proporcao=grouped_data["size"] / grouped_data["size"].sum())
sns.barplot(data= grouped_data, x="Rating", y="Proporcao")
plt.xlabel("Avaliação")
plt.ylabel("Proporção");


Notamos assim uma disputa entre amor e ódio em relação ao aplicativo, portanto agora é importante refletir sobre alguns pontos:

* Existe algum padrão no uso de palavras para esses *reviews*? 
* Há algum momento no tempo que há mais *reviews* "negativos" que "positivos"? 
* É possível classificar o sentimento do autor somente baseado no texto?


Nas seguintes seções vamos aprofundar a análise desses *reviews* em relação ao tempo,  também aprofundaremos ainda mais a análise em relação aos *reviews* considerando também o texto deixado pelo autor, dessa maneira, as respostas para essas perguntas vão surgir ao longo do texto.

\subsection{Analisando os Review de Maneira Temporal}


In [ ]:
df['Time_submitted'] = pd.to_datetime(df['Time_submitted'])

#%%
df['month'] = df['Time_submitted'].apply(lambda x: x.month)
df['hour'] = df['Time_submitted'].apply(lambda x: x.hour)
df['day'] = df['Time_submitted'].apply(lambda x: x.day)

Vamos nessa subseção averiguar os *review* em termos de contagens e valores médios referentes, a mês, dia e hora.

\subsubsection{Review Ao longo dos Meses}

O que pode se notar pela @fig-plt é um comportamento elevado de *reviews* no mês 4 e o que acarretou em um decréscimo no valor médio dos *reviews* no mesmo mês. 


In [ ]:
#| label: fig-plt
#| fig-cap: Comportamento dos Reviews ao longo dos meses
time_plot(df, 'month')

\subsubsection{Review Ao longo dos Dias}
Em relação aos dias temos um comportamento semelhantes aos meses, em que dias que possuem mais *reviews*, possuem avaliação média inferior aos demais, isso indica portanto que o ódio nasce quando as pessoas se reúnem^[Ou é ao contrário?].

In [ ]:
#| fig-cap: "Comportamento dos Reviews ao longo dos dias"

time_plot(df, 'day')

\subsubsection{Review Ao longo das Horas}

Em relação as horas temos uma grande quantidade de *reviews* no horário das 10-15h e depois o pico acontece as 18h, assim como o menor valor de *review* também acontece as 18h. Vale ressaltar que a média em relação as horas tende a manter-se estacionária até as 15h, um comportamento não visto na média dos meses e dias.

In [ ]:
#| fig-cap: "Comportamento dos Reviews ao longo das horas"

time_plot(df, 'hour')

\section{\centering Texto  e seus ajustes}

Em toda análise textual é necessária um bom pré processamento dos dados, portanto é isso que vamos tratar nessa seção. Realizaremos:

* Remoção de Stop Words^[Stop Words são palavras as quais possuem uma alta taxa de frequência no texto, mas carregam pouca informações para a análise a ser realizada.]

* Lematização^[Tarefa de determinar se duas palavras tem a mesma raiz, apesar de diferença de "estrutura"]

* Remoção de Espaços indevidos

Porém antes do processamento, vejamos o comportamento da frequência das palavras:
Para o casos de 5 estrelas notamos palavras positivas como "love", "great"  e "easy" em destaque.


In [ ]:
#| fig-pos: 'h'
#| fig-cap: "Wordcloud para Reviews de 5 estrelas"
my_wc(df, 4)

E para o caso de reviews de 1 estrela nota-se palavras referentes a problemas como "fix", "even" e "issue".


In [ ]:
#| fig-pos: 'h'
#| fig-cap: "Wordcloud para Reviews de 1 estrela"
my_wc(df, 'podre')

Nota-se em ambos os casos que palavras como "Spotify", "song", "app" e "music" possuem uma alta frequência de exibição, entretanto não agregam muita informação para a análise, assim é interessante a remoção delas.

\subsubsection{Um ajuste dos Reviews}
Vamos aqui nessa subseção recategorizar a variável numérica da avaliação deixando da seguinte maneira:

* 1 a 2 Estrelas: Negativo
* 3 Estrelas: Neutro
* 4 a 5 Estrelas: Positivo

Portanto a partir desse momento, quando refere-se a review negativos, estamos nos referenciando a 1 a 2 estrelas e assim por diante.


In [ ]:
df['Review_words'] = [len(x.split()) for x in df['Review']]
rating = df['Rating'] 
df['Sentiment'] = np.where((1 <= rating) & (rating <= 2), 'negativo', 
np.where((4 <=rating) & (rating<=5), 'positivo', 'neutral'))

\subsection{Análise Sem as Stop Words}

Nessa breve subseção remover as **stops words**, fazer algumas análises em relação aos *reviews* agora categorizados e também refazer algumas análises da seção anterior. Além das stop words "tradicionais" do inglês, serão removidas as seguintes palavras: app, song, music, e spotify.


In [ ]:
stop_words = set(stopwords.words("english"))
stop_words.update(["app", "song", "Spotify", "music", "spotify"])

df['Review_wsw'] = df['Review'].apply(lambda x: " ".join(x for x in x.split() if x not in stop_words))

Vejamos a titulo de curiosidade a wordcloud para *reviews* de 5 estrelas, agora sem stop words.


In [ ]:
#| fig-cap: "Wordcloud para Reviews de 5 estrelas sem Stop Words"
my_wc(df, var = "Review_wsw", Filter = 4)

Nota-se, agora um destaque para palavras positivas como: "love", "great" e "good".


## Palavras em Review
Nessa subseção vamos averiguar a quantidade de palavras pelos *reviews*, será que *reviews* positivos possuem mais palavras que negativos? Será que o comportamento da quantidade das palavras se mantém constante ao longo do tempo? As respostas para essas perguntas e mais serão respondidas nessa seção.

### Para Reviews Negativos
Para o caso de *reviews* negativos, notamos uma média de aproximadamente 38 palavras, um mínimo de 3 palavras, e um *review* que incrivelmente atinge 700 palavras!

Vale que salientar que apesar da existência de reviews bastante tagarelas, a quantidade de palavras tende a se comportar a sua maior parte até 100, como podemos ver tanto pela @tbl-neg e pela @fig-neg.

In [ ]:
#| label: tbl-neg
#| tbl-cap: Análise descritiva para a quantidade de palavras em sentimentos negativos
Display(df[df['Sentiment'] == 'negativo'][['Review_words']].rename(columns={"Review_words": "Total de Palavras"}).describe().T.to_markdown())

In [ ]:
#| label: fig-neg
#| fig-cap: Quantidade de palavras para sentimentos negativos
sns.boxplot(data=df[df["Sentiment"] == 'negativo'], x="Review_words")
plt.xlabel("Quantidade de Palavras");


### Para Reviews Positivos
Referente aos *reviews* positivos temos um hábito bastante diferente do que os negativos, pois  temos o quantil de 75% de somente 31 palavras, enquanto no negativo eram 50, a mediana é de 18 para casos positivos enquanto para negativos era de 32. Pela @fig-pos podemos reparar muitos *reviews* categorizados como *outliers* isso se deve ao fato de que a maior parte possui poucas palavras.


In [ ]:
#| label: tbl-pos
#| tbl-cap: Análise descritiva para a quantidade de palavras em sentimentos positivos
Display(df[df['Sentiment'] == 'positivo'][['Review_words']].rename(columns={"Review_words": "Total de Palavras"}).describe().T.to_markdown())

In [ ]:
#| label: fig-pos
#| fig-cap: Quantidade de palavras para sentimentos positivos
sns.boxplot(data=df[df["Sentiment"] == 'positivo'], x="Review_words")
plt.xlabel("Quantidade de Palavras");

## Revisitando o tempo
Vamos nessa subseção averiguar o comportamento da quantidade de palavras por sentimento ao longo do tempo. Assim o que podemos notar pela @fig-tmp é que os *reviews* classificados como positivos tem uma quantidade de palavras bem inferior aos negativos não somente no sentido global como visto pela @tbl-neg e @tbl-pos mas também no sentido temporal.

Em relação ao comportamento em relação aos dias, tem-se que para os *reviews* positivos visualmente uma estacionariedade em relação a quantidade de palavras, o que não acontece para o caso de *reviews* neutros e negativos. Para o caso dos meses é visível a não estacionariedade, tem-se que para meses iniciais do ano, os *reviews* possuem mais palavras que os situados no meses centrais do ano.


In [ ]:
#| label: fig-tmp

#| fig-cap: "(a) Total de Palavras por Review ao longo dos dias, (b) Total de Palavras por Review ao longo dos meses"


fig, ax = plt.subplots(1, 2)
ax[0].set_title("(a)")
ax[0].set_ylabel("Total")
ax[1].set_ylabel("Total")
ax[1].set_title("(b)")
sns.lineplot(data=df[df['Review_words']< 180], y='Review_words', x='day', hue='Sentiment', ax=ax[0])
ax[0].get_legend().remove()
sns.lineplot(data=df[df['Review_words']< 180], y='Review_words', x='month', hue='Sentiment', ax=ax[1])
fig.show()

\section{\centering Modelagem}
Vamos nessa seção criar um modelo preditivo que fornece o sentimento do autor do *review* baseado no texto deixado pelo mesmo. Reiterando que um sentimento positivo nesse contexto, significa *review* numérico de 4 a 5 estrelas, e um sentimento negativo *reviews* de 1 a 2 estrelas na Play Store.


In [ ]:
data = df[df['Sentiment'] != 'neutral']
## MODELAGEM
X = data['Review_wsw']
y = data['Sentiment']

\subsection{Desenvolvendo as Covariáveis}
Como vimos, não temos covariáveis "automáticas" no **dataset**, assim faz-se necessário a criação baseando-se no *review* escrito, vamos aqui utilizar de duas técnicas, a saber Vetores  de Contagens^[Count Vectorization] e TF-IDF. Dessa forma pretendemos avaliar o sentimento do usuário ao fazer o *review* escrito, por questões técnicas^[Técnicas aqui quer dizer: Não foi apresentado no curso a qual esse trabalho esta sendo apresentado] vamos utilizar somente os *reviews* que são positivos e negativos, dessa maneira removendo os classificados como neutros, tornando-se um problema de classificação binária.
Vejamos pela @fig-pie a proporção de *reviews* agora categorizados como negativos e positivos.


In [ ]:
#| label: fig-pie
#| fig-cap: Proporção das Avaliações por Sentimento
#| fig-pos: H
plt.pie(x=y.value_counts(), labels=y.unique(), autopct='%.0f%%');

Para a tristeza daqueles que tendem a inclinar-se a suspeitar mais do mal que do bem, temos agora uma quantidade levemente superior para casos de *reviews* de sentimentos positivos, dessa forma em termos de modelagem, é dito um problema de classes^[Onde classes refere-se : Positivo e Negativo] balanceadas.


\subsubsection{Count Vectorization}
A ideia é bastante simples, vamos criar uma coluna/vetor para cada palavra única no banco de dados e associar a quantidade de vezes que aparece no *review* escrito, um exemplo didático pode ser visto a seguir.

![Exemplo de CV](docs/imgs/vector.jpg){fig-align="center"}


In [ ]:
vectorizer = CountVectorizer()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=13)

In [ ]:
vectorizer.fit(X_train)
X_train_count = vectorizer.transform(X_train)
X_test_count = vectorizer.transform(X_test)

\subsubsection{TF-IDF}

O termo TF-IDF significa "Term Frequency - Inverse Document Frequency", é uma técnica assim como Count Vectorizer (CV) para contar a quantidade de palavras em um documento^[Documento no contexto de NLP signfica observação], ao contrário do CV, aqui utilizamos *scores* para cada palavra, que, em geral, refere-se a relevância dela no documento.

O método para computar o TF-IDF é feito multiplicando duas métricas:

* Quantas vezes a palavra apareceu no documento (tf)
* Inverso da frequência da palavra entre os documentos (idf)

Em termos formais temos:
Seja w uma palavra, d um documento, e D o conjunto dos documentos^[Também conhecido como corpus], o tf-idf fica definido como:

$$tfidf(w, d, D) = tf(w, d)idf(t, D)$$

In [ ]:
tfidf = TfidfVectorizer(analyzer='char', ngram_range=(2,3))
tfidf.fit(X_train)
X_train_tfidf = tfidf.transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

\subsection{Ajustando Modelos}
Vamos aqui ajustar uma regressão logística e um modelo de random forest para classificar o sentimento do texto. Desse modo vamos ajustar 4 modelos, pois vamos ajustar também um modelo para CV e um modelo para TF-IDF


\subsubsection{Modelos Com CV}
Pela @fig-models somos capazes de constatar que apesar de CV ser um método ingênuo, ele é capaz de criar covariáveis que conseguem predizer com uma acurácia extremamente adequada nos dados de teste o sentimento do autor do *review* em ambos os modelos.


In [ ]:
rf_cv = RandomForestClassifier(random_state=19, n_estimators=105, min_samples_split=12, min_samples_leaf=1)
rf_cv.fit(X_train_count, y_train)
rf_cv_accuracy = rf_cv.score(X_test_count, y_test)

In [ ]:
rl_cv = LogisticRegression(random_state=0, penalty='none', max_iter=10000)
rl_cv.fit(X_train_count, y_train)
rl_cv_score = rl_cv.score(X_test_count, y_test)

In [ ]:
#| label: fig-models

#| fig-cap: "(a) Curva Roc para a Random Forest usando CV, (b)  Curva Roc para a 
# Regressão Logistica usando CV"
fig, ax = plt.subplots(1,2)
ax[0].set_title("(a)")
ax[1].set_title("(b)")
RocCurveDisplay.from_estimator(rf_cv, X_test_count, y_test, ax=ax[0])
RocCurveDisplay.from_estimator(rl_cv, X_test_count, y_test, ax=ax[1]);

\subsubsection{Modelos Com TF-IDF}

Apesar da técnica TF-IDF ser mais meticulosa que o CV, o seu esforço em criar covariáveis não condiz com a sua performance na detecção dos sentimentos, visto que em ambos os modelos, tem desempenho inferior aos modelos com CV.^[A realidade pode ser dolorosa, mas é algo que você deve aprender a aceitar.]


In [ ]:
rf_tf = RandomForestClassifier()
rf_tf.fit(X_train_tfidf, y_train)
rf_tf_score = rf_tf.score(X_test_tfidf, y_test)

In [ ]:
rl_tf = LogisticRegression(random_state=0, penalty='none', max_iter=10000)
rl_tf.fit(X_train_tfidf, y_train)
rl_tf_score = rl_tf.score(X_test_tfidf, y_test)

In [ ]:
#| label: fig-models-tf

#| fig-cap: "(a) Curva Roc para a Random Forest usando CV, (b)  Curva Roc para a Regressão Logistica usando CV"
fig, ax = plt.subplots(1,2)
ax[0].set_title("(a)")
ax[1].set_title("(b)")
RocCurveDisplay.from_estimator(rf_tf, X_test_tfidf, y_test, ax=ax[0])
RocCurveDisplay.from_estimator(rl_tf, X_test_tfidf, y_test, ax=ax[1]);

\subsubsection{Modelo Final}
Finalizando, temos como modelo final uma random forest com pré processamento de CV, com 25% das observações sendo para teste, obteve aproximadamente uma acurácia de 88%.




# O começo e o fim são um só


Um fato a ser considerado é que a existência do *review* escrito é dependente do review numérico, assim a análise anterior somente teria validade, se por algum motivo fosse possível deixar somente o review escrito, o que no momento não acontece.

Assim, é necessário extrair o sentimento do autor do texto **exclusivamente** pelo texto, pois dessa maneira podemos conflitar a predição do sentimento com relação a estrela deixada como *review*. Visto que existe um fenômeno tal qual o *review*  numérico não condiz com o sentimento do autor, pois como já dizia Dostoievski "A natureza das pessoas não são todas iguais; para muitas, uma conclusão lógica transforma-se às vezes num sentimento fortíssimo que domina todo o seu ser e que é muito difícil de expulsar ou transformar...".


# Epilógo

Vamos aqui introduzir um modelo mais robusto para analisar os sentimentos, vamos conflitar a classificação com o review numérico, e juntamente comparar com as predições do modelo anteriormente utilizado.

## Modelo Vader

Valence Aware Dictionary for Sentiment Reasoning, ou Vader é um modelo de classificação de sentimentos semi-supervisionado onde é possível analisar ambas^[Positivo e Negativo] polaridades de um texto, Vader é capaz de fornecer não somente o sentimento, mas também a intensidade do mesmo.

O modelo possui uma grande vantagem sobre os que foram utilizados aqui, pois não é necessário pré-processamento do texto e ele também consegue lidar bem com pontuações, emojis e palavras em caixa baixa/alta.

O Vader nos fornece a intensidade de sentimento positivo, negativo, neutro e também um escore denominado *compound* que normaliza^[A medida fica entre -1 e 1] os três sentimentos anteriores em um só, sendo valores mais próximos de -1 indicando sentimento negativo e mais próximos de 1 sentimentos positivos.


Vejamos dessa maneira pela  @tbl-vad a comparação da Random com CV e do Vader.

In [ ]:
def my_predict(text):
    teste = vectorizer.transform(pd.Series(text))
    return rf_cv.predict(teste)
#%%
def predict_vader(text):
    sid = SentimentIntensityAnalyzer()
    sentiment = sid.polarity_scores(text)
    return sentiment['compound']



#%%
predict_vader(X_test.reset_index(drop=True)[3])

#%%
my_predict(X_test.reset_index(drop=True)[3])[0]

#%%

texts = [X_test.reset_index(drop=True)[3],
X_test.reset_index(drop=True)[21],
X_test.reset_index(drop=True)[35],
X_test.reset_index(drop=True)[40],
X_test.reset_index(drop=True)[47]]

#%%
compar_models = pd.DataFrame({"texts": texts})


#%%
compar_models['Random Forest'] = compar_models.apply(my_predict)

compar_models['Vader'] = compar_models['texts'].apply(predict_vader)
#%%

compar_models["Vader Resumido"] = compar_models["Vader"].apply(lambda x: np.where(x > 0, 'positivo',
np.where(x < 0, 'negativo', 'neutro')))

# %%
real_values = [y_test.reset_index(drop=True)[3],
y_test.reset_index(drop=True)[21],
y_test.reset_index(drop=True)[35],
y_test.reset_index(drop=True)[40],
y_test.reset_index(drop=True)[47]]

compar_models["Sentimento Real"] = real_values

In [ ]:
#| label: tbl-vad
#| tbl-cap: Comparação dos Modelos
Display(compar_models.drop('Vader', axis=1).to_markdown())

O que podemos notar é que em casos que o Vader classifica o *review* como neutro (*compound* = 0) a random forest classifica como negativo, e no ocorrências em que o Vader não classifica como neutro temos concordância entre os modelos.

# Referências
